# Lab 12 - Decision Tree regression

In this lab, we will return to the health insurance dataset used in Labs 7 and 9, and use *decision trees* to try to predict the policy cost, given information about the policy holder.  A decision tree is essentially a flow chart that is followed to make the predictions.

If you haven't already done so, watch the video introducing decision trees.

### Section 1: Installing the graphviz package

Follow the appropriate instructions for how you are running Python and Jupyter notebooks.

**Jupyter Hub on Lehman 360:**

1) Run the following code and wait until the [*] beside the cell changes to a number to continue (may take 5-10 min; if takes longer, try restarting the notebook).

2) Once graphviz is installed, click on "Kernel" in the top menu and then on "Restart" to restart the notebook.

In [ ]:
!pip install --user graphviz

#### Google Colab:
The graphviz package should be installed.

#### Anaconda on your own computer:

Install graphviz the same way you installed folium.  

##### Instructions given in Lab 6:

1. In Anaconda Navigator, click on Environment in the left menu.
2. Select "Not installed" in the list.
3. Select folium and click "Apply" in the pop-up box.

[Instructions with images](https://docs.anaconda.com/anaconda/navigator/tutorials/manage-packages/#installing-a-package)

##### Install from the prompt/terminal:

Alternatively, on Windows: open the Anaconda Prompt (should be listed along with Jupyter Notebook) 

or on Mac or Linux: open the Terminal (on Mac under Applications -> Utilities -> Terminal)

Type and run the following in the prompt or terminal: 

`conda install -c anaconda graphviz`

### Section 2: Loading and cleaning the data

Run the following import statements.  If you get an error about graphviz, the package was not installed properly.  See above instructions and/or contact me for help.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import graphviz
 
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

%matplotlib inline

Insurance data URL:  [https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv](https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv)

In this data, each row represents an insurance policy and the 7 columns contain the following information about it:
- age: age of policy holder
- sex: sex of policy holder
- bmi: boday mass index (bmi) of policy holder.  bmi is a (sometimes unreliable) measurement of body fat in adults
- children: number of children (dependents) on the policy
- smoker: whether the policy holder is a smoker
- region: region of the country the policy holder lives in
- charges: price for insurance policy

Load the insurance data into the DataFrame `insurance`:

Decision trees can only use numeric data as the independent variables or *features*, so we have to convert the `sex`, `smoker`, and `region` columns into dummy variables as in Lab 9.

<details><summary>Answer:</summary>
<code>
insurance = pd.get_dummies(insurance, columns = ["sex", "smoker", "region"], drop_first = True)
    </code>
</details>

Display your new DataFrame to make sure the conversion was done correctly.

### Section 3:  Fitting a decision tree with scikit-learn

To prepare the data for use by scikit-learn, we have to create a new DataFrame of only the independent variables or features, and a column of only the dependent variable, which we want to predict.  We also did this in Section 5 of Lab 9 to use scikit-learn to run linear regression on our data. 

First, create a new DataFrame called `x` of only the independent variables. 

Next create a Series called `y` of only the `charges` column.

Now we create the decision tree regressor variable, called `model`, and fit it to our data:

In [ ]:
model = DecisionTreeRegressor(max_depth = 3)
model.fit(x, y)

If you are running this notebook on your own computer or in Google Colabs, you may be able to display the decision tree by first importing the function `plot_tree` and then running it.

In [ ]:
# will give error in Jupyter Hub on Lehman 360
from sklearn.tree import plot_tree

In [ ]:
# will give error in Jupyter Hub on Lehman 360

# following line of code can be used to adjust the figsize to make it readable
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (5,5), dpi=300)
plot_tree(model)

If you are using Jupyter Hub on Lehman 360, run the following code (which will give an error):

In [ ]:
dot_data = export_graphviz(model, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("insurance.dot")

Despite the error, there should now be a file called `insurance.dot` in your directory.  To view the fitted decision tree, open the `insurance.dot` file in Jupyter and copy the text.  Paste this text into the text box at [http://www.webgraphviz.com](http://www.webgraphviz.com) and click the "Generate graph!" button at the bottom.

The column names have been replaced by `X[0], X[1], ..., X[7]`.  To change these to the actual column names do the following:

* If `plot_tree()` worked, add the parameter `feature_names= x.columns` to `plot_tree()` and re-run the code.



* If you created a `.dot` file, run the following code to create a new file called `insurance_fixed.dot` from `insurance.dot` that has `X[0], X[1], ..., X[7]` replaced by the original column names.  As with `insurance.dot`, copy the contents of `insurance_fixed.dot` in the text box at [http://www.webgraphviz.com](http://www.webgraphviz.com) to view the new tree.

In [ ]:
with open ("insurance.dot", "r") as fin:
    with open("insurance_fixed.dot","w") as fout:
        for line in fin.readlines():
            line = line.replace("X[0]","age")
            line = line.replace("X[1]","bmi")
            line = line.replace("X[2]","children")
            line = line.replace("X[3]","sex_male")
            line = line.replace("X[4]","smoker_yes")
            line = line.replace("X[5]","region_northwest")            
            line = line.replace("X[6]","region_southeast")
            line = line.replace("X[7]","region_southwest")
            fout.write(line)

Which column is the first decision based on?  Does this make sense?

Which columns are the decisions in the next row based on?

How many levels are in the tree?  How do you think this relates to the tree having a max depth of 3?

What value is predicted by the decision tree for someone who is a smoker, age 50, and has a bmi of 27?

What value is predicted by the deicsion tree for someone who is not a smoker, age 37, and has a bmi of 23?

What happens if you change the `max_depth` parameter to 5 in DecisionTreeRegressor?

Look at the leaves of the tree.  What's the smallest sample?  Are the sample sizes somewhat evenly distributed?  

Very small sample sizes in some of the leaves can be a sign of overfitting.  In a decision tree, a too large max depth leads to overfitting, while a too small max depth leads to underfitting.


### Section 4: Training and testing data

In Lab 10, we simulated different training and testing datasets to look for overfitting by comparing the mean squared error (MSE).  We cannot simulate more insurance data, but we can divide our existing dataset into two parts:  a training dataset and a testing dataset.  We will train our model on only the training data, and then test it on only the testing data.

The `train_test_split` function in scikit-learn will randomly divide our data into two parts.  The parameter `test_size` tells scikit-learn what proportion of the original data should becomes test data.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

What do you think is was assigned to the variables `x_train`, `x_test`, `y_train`, and `y_test`?  Display them below to see if you are correct.

Create a decision tree with `max_depth = 3` from the training data:

<details><summary>Answer:</summary>
<code>
reg3 = DecisionTreeRegressor(max_depth = 3)
reg3 = reg3.fit(x_train, y_train)
</code>
</details>

Make predictions for the test data:

<details><summary>Answer:</summary>
<code>
predictions_3 = reg3.predict(x_test)
</code>
</details>

Compute the mean squared error for these predictions:

<details><summary>Answer:</summary>
<code>
mean_squared_error(predictions_3, y_test)
</code>
</details>

Next make predictions using the training data and compute the mean squared error for these predictions.

How do the two mean squared errors compare?  If the mean squared error of the testing data is a lot higher than the mean squared error of the training, then this is a sign that overfitting is happening.

Write down or copy the mean squared errors into a blank cell to "store" them.  Now try re-running the above code with a smaller max depth.  How do the training and testing mean squared errors change?  What do you think this means?

Next re-run the above code with a larger max depth. How do the training and testing mean squared errors change?  What do you think this means?

What do you think the best max depth is for this data?  We will compare the decision tree model with that max depth to a linear regression model in the next section.

### Section 5:  Comparison to Linear Regression

Finally, let's determine with a decision tree or linear regression does a better job at predicting health insurance policy costs.

Using the same training and testing data as above, fit a linear regression model (using scikit-learn) to the training data.

Use this linear regression model to make predictions for the test data.

Compute the mean squared error of these linear regression model predictions.

How does the mean squared error of the linear model for the test data compare to the mean squared error of the best decision tree model picked in Section 4?  

Which model to you think makes the best predictions?